# Exploring ways of finding hair in images
This is an exploratory notebook - it doesn't really go in order, but I wanted to save some of the things I tried, for posterity. I did not use anything in this notebook for the final product.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pymeanshift as pms
import os
from glob import glob
from PIL import Image 

## Resize images
This function takes the images and resizes them to 300x400 pixels.

In [ ]:
imgs = glob('./hair_images/*')

def resize(imgs):
    for img in imgs:
        filename = img.split('/')[2]
        filename = filename.split('.')[0]
        if filename == 'image_urls':
            continue
        if os.path.exists('./resized_hair_images/'+filename+'_resized.jpg') == True:
            continue
        im = Image.open(img)
        imResize = im.resize((300,400), Image.ANTIALIAS)
        imResize.save('./resized_hair_images/'+filename+'_resized.jpg', 'JPEG', quality=90)

resize(imgs)

## Find hair via contours
This code explores using skin tones to find adjacent hair.

In [ ]:
min_YCrCb = np.array([0,133,77],np.uint8)
 max_YCrCb = np.array([255,173,127],np.uint8)

In [ ]:
img = cv2.imread('wavy.jpg')

imageYCrCb = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contour on the source image
for i, c in enumerate(contours):
    area = cv2.contourArea(c)
    if area > 9000:
        cv2.drawContours(img, contours, i, (0, 255, 0), 3)

# Display the source image
cv2.imwrite('output1.jpg', img)
plt.figure(figsize=(12,12))
plt.imshow(img[:,:,::-1])
plt.show()

## Find faces
Using the the built-in Haar Cascades in OpenCV to find faces.

In [ ]:
face_cascade = cv2.CascadeClassifier('/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/cv2/data/haarcascade_eye.xml')

In [ ]:
img = cv2.imread('hair.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3)

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray,minSize=(100, 100), maxSize=(150,150))
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

cv2.imwrite('output_hair.jpg', img)
plt.figure(figsize=(12,12))
plt.imshow(img[:,:,::-1])
plt.show()

In [ ]:
img = cv2.imread('hair.jpg')
img = cv2.cvtColor(img ,cv2.COLOR_BGR2GRAY)
cv2.imwrite('blackwhite.jpg', img)

(segmented_image, labels_image, number_regions) = pms.segment(img, spatial_radius=6, 
                                                              range_radius=4.5, min_density=1500)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(segmented_image)
plt.show()

cv2.imwrite('blackwhite_seg.jpg', segmented_image)

## Playing with edge detection
This didn't work very well.

In [ ]:
img = cv2.imread('hair_images/903ff49749.jpg',0)
edges = cv2.Canny(img,100,250)

plt.figure(figsize=(20,20))
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
     
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
 
    # return the edged image
    return edged

imagePath = 'hair_images/903ff49749.jpg'
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)


In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
 
# apply Canny edge detection using a wide threshold, tight
# threshold, and automatically determined threshold
wide = cv2.Canny(blurred, 10, 200)
tight = cv2.Canny(blurred, 225, 250)
auto = auto_canny(blurred)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(wide,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()